Данная программа создана для Python3

## Что необходимо для работы с данной программой
Вобщем то не так уж и много. Сама программа предназначена для пакетного преобразования файлов, полученных с осциллографа Актаком, из формата .bin в формата .xlsx. 
Для работы необходим paths.xlsx файл, в котором в первой ячейке стоит 0, а дальше в этом же столбце идут подряд пути к папкам, в которых находятся .bin файлы. Папок (как и путей к ним) может быть сколько угодно, количество .bin файлов в папке также не ограничено. Программа автоматически обрабатывает все .bin файлы, находящиеся по указанному пути, поэтому будьте внимательны, чтобы в данной папке не оказался посторонний .bin файл. Выходные файлы имеют такое же название, как и исходный .bin файл. 
#### Внимание!
Полученные таблицы формата .xlsx отличаются по формату расположения данных от таблиц .xls, полученных с помощью родной актакомовской программы. Возможно возникновение ошибок в случае последующего использования их в других программах обработки данных.

### Импорт библиотек

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import struct
import os

### Определение функций

In [3]:
def openBIN(name): # Функция, открывающая .bin файл
    data = open('%s' %name, 'rb').read()
    return data[10:] # Вырезаем первые 10 символов, которые не нужны для последующей обработки

def bitEncode(data): # Декодируем данные из байтового формата в целочисленный со знаком
    encodeData = np.zeros(int(len(data) / 2))
    for i in range(int(len(data) / 2)):
        encodeData[i] = int.from_bytes(data[2 * i:2* i + 2], 'little', signed=True)
    return encodeData

def dataInfo(data): # Извлечение основной информации из .bin файла
    # Стоит отметить, что большинство информации, находящейся в .bin файле необходимо для посттроения графиков
    # в родной актакомовской программе и не является важной для перевода в эксельку.
    if (len(data) - 3 == int.from_bytes(data[3:5], 'little')): # Случай 1 канала записи
        info = pd.DataFrame([1, int.from_bytes(data[11:13], 'little'), 
                             struct.unpack("f", data[47:51])[0] * 10, str(data[:3])[2:5]])
        # число каналов; количество реальных точек; название канала (CH1 или CH2); частота дискретизации в мВ
    else: # Случай 2 каналов записи
        # endCH1 - количество байт в первом канале (необходимо, чтобы найти начало второго канала, который также начинается с основной информации)
        endCH1 = int.from_bytes(data[3:5], 'little') + 3 
        info = pd.DataFrame([2, int.from_bytes(data[11:13], 'little'), struct.unpack("f", data[47:51])[0] * 10, str(data[:3])[2:5],
                             endCH1, int.from_bytes(data[endCH1 + 11:endCH1 + 13], 'little'),
                            struct.unpack("f", data[endCH1 + 47: endCH1 + 51])[0] * 10, str(data[endCH1:endCH1 + 3])[2:5]])
        # число каналов; количество реальных точек 1-го канала; название 1-го канала канала (CH1 или CH2);
        # частота дискретизации 1-го канала в мВ; количество байт в первом канале; количество реальных точек 2-го канала;
        # название 2-го канала канала (CH1 или CH2); частота дискретизации 2-го канала в мВ.
    return info

def channels(data, info): # Возвращает массивы данных в каналах
    if (info[0][0] == 1):
        return bitEncode(data[51: 51 + info[0][1] * 2]).T * info[0][2]
    else:
        return np.array([bitEncode(data[51:51 + info[0][1] * 2]) * info[0][2],
                        bitEncode(data[info[0][4] + 51: 51 + info[0][4] + info[0][1] * 2]) * info[0][6]]).T

def finalData(inpArr, info): # Переделывает массивы входных данных в dataframe для последующей записи в эксельку
    if (info[0][0] == 1):
        CH = pd.DataFrame(inpArr, columns=[info[0][3]])
    else:
        CH = pd.DataFrame(inpArr, columns=[info[0][3],info[0][7]])
    return CH

def binFiles(path): # Возвращает массив названий всех .bin файлов, находящихся в директории path
    a = os.listdir(path)
    b = []
    for i in a:
        if i[-3:] == 'bin':
            b.append(i) 
    return b

### Основной программный блок

In [4]:
# В случае использования данной программы замените путь ниже на расположение данной программы.
os.chdir('C:\\Users\\Yaroslav\\YandexDisk\\Pr\\Py\\lab\\openBin')
paths = pd.read_excel('paths.xlsx')
for path in paths[0]:
    listBins = binFiles(path)
    for binfile in listBins:
        os.chdir(path)
        data = openBIN(binfile)
        info = dataInfo(data)
        outData = finalData(channels(data, info), info)
        os.chdir(path)
        outData.to_excel('%s.xlsx' %binfile[:-4])